In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import copy

sns.set_style("darkgrid")
sns.mpl.rc("figure", figsize=(25, 5))
sns.mpl.rc("font", size=14)

!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.0 MB/s eta 0:00:00


In [ ]:
# upload train and test data here
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
data = train.append(test, ignore_index=True)
data.sample(5)

,Store_id,SKU_id,Date,Promo,Demand,Regular_Price,Promo_Price,id
12747,26,1,17.02.2015,1.0,961.0,149.37,143.8,NaN
56615,26,2,02.07.2015,NaN,0.0,150.73,NaN,NaN
85645,91,2,21.02.2016,NaN,3.0,138.36,NaN,NaN
37129,74,1,16.04.2016,NaN,28.0,135.78,NaN,NaN
89241,84,1,24.05.2016,NaN,NaN,128.98,NaN,2325.0


In [ ]:
from ipywidgets import IntProgress
from itertools import product
from copy import deepcopy

def percentile(n):
    '''Calculate n - percentile of data'''
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'pctl%s' % n
    return percentile_
    
# add missing dates to GroupBy.Core object
def fill_missing_dates(x, date_col):
    min_date, max_date = x[date_col].min(), x[date_col].max()
    groupby_day = x.groupby(pd.PeriodIndex(x[date_col], freq='D'))
    results = groupby_day.sum(min_count=1)

    idx = pd.period_range(min_date, max_date)
    results = results.reindex(idx, fill_value=np.nan)

    results.index.rename(date_col, inplace=True)

    return results


def calc_preag_fill(data, group_col, date_col, target_cols, preagg_method):
    ## calc preaggregation
    data_preag = data.groupby(group_col).agg(
        preagg_method)[target_cols].reset_index()

    ## fill missing dates
    data_preag_filled = data_preag.groupby(group_col[:-1]).apply(
        fill_missing_dates, date_col=date_col).drop(group_col[:-1],
                                                    axis=1).reset_index()

    ## return DataFrame with calculated preaggregation and filled missing dates
    return data_preag_filled


def calc_rolling(data_preag_filled, group_col, date_col, method, w):

    ## calc rolling stats
    lf_df_filled = data_preag_filled.groupby(group_col[:-1]).\
        apply(lambda x: x.set_index(date_col).rolling(window=w, min_periods=1).agg(method)).drop(group_col[:-1], axis=1)

    ## return DataFrame with rolled columns from target_vars
    return lf_df_filled


def calc_ewm(data_preag_filled, group_col, date_col, span):
    ## calc ewm stats
    lf_df_filled = data_preag_filled.groupby(group_col[:-1]).\
        apply(lambda x: x.set_index(date_col).ewm(span=span).mean()).drop(group_col[:-1], axis=1)

    ## return DataFrame with rolled columns from target_vars
    return lf_df_filled


def shift(lf_df_filled, group_col, date_col, lag):

    lf_df = lf_df_filled.groupby(
        level=group_col[:-1]).apply(lambda x: x.shift(lag)).reset_index()
    lf_df[date_col] = pd.to_datetime(lf_df[date_col].astype(str))

    ## return DataFrame with following columns: filter_col, id_cols, date_col and shifted stats
    return lf_df


def generate_lagged_features(
        data: pd.DataFrame,
        target_cols: list = ['Demand'],
        id_cols: list = ['SKU_id', 'Store_id'],
        date_col: str = 'Date',
        lags: list = [7, 14, 21, 28],
        windows: list = ['7D', '14D', '28D', '56D'],
        preagg_methods: list = ['mean'],
        agg_methods: list = ['mean', 'median', percentile(10), pd.Series.skew],
        dynamic_filters: list = ['weekday', 'Promo'],
        ewm_params: dict = {'weekday': [14, 28], 'Promo': [14, 42]}) -> pd.DataFrame:
    
    ''' 
    data - dataframe with default index
    target_cols - column names for lags calculation
    id_cols - key columns to identify unique values
    date_col - column with datetime format values
    lags - lag values(days)
    windows - windows(days/weeks/months/etc.),
        calculation is performed within time range length of window
    preagg_methods - applied methods before rolling to make
        every value unique for given id_cols
    agg_methods - method of aggregation('mean', 'median', percentile, etc.)
    dynamic_filters - column names to use as filter
    ewm_params - span values(days) for each dynamic_filter
    '''

    data = data.sort_values(date_col)
    out_df = deepcopy(data)
    dates = [min(data[date_col]), max(data[date_col])]

    # total = len(target_cols) * len(lags) * len(windows) * len(preagg_methods) * len(agg_methods) * len(dynamic_filters)
    # progress = IntProgress(min=0, max=total)
    # display(progress)

    for filter_col in dynamic_filters:
        group_col = [filter_col] + id_cols + [date_col]
        for lag in lags:
            for preagg in preagg_methods:
                data_preag_filled = calc_preag_fill(data, group_col, date_col,
                                                    target_cols, preagg)

                ## add ewm features
                for alpha in ewm_params.get(filter_col, []):
                    ewm_filled = calc_ewm(data_preag_filled, group_col,
                                          date_col, alpha)
                    ewm = shift(ewm_filled, group_col, date_col, lag)
                    new_names = {x: "{0}_lag{1}d_alpha{2}_key{3}_preag{4}_{5}_dynamic_ewm".\
                        format(x, lag, alpha, '_'.join(id_cols), preagg, filter_col) for x in target_cols}

                    out_df = pd.merge(out_df,
                                      ewm.rename(columns=new_names),
                                      how='left',
                                      on=group_col)
                
                ## add rolling features
                for w in windows:
                    for method in agg_methods:
                        rolling_filled = calc_rolling(data_preag_filled,
                                                      group_col, date_col,
                                                      method, w)

                        ## lf_df - DataFrame with following columns: filter_col, id_cols, date_col, shifted rolling stats
                        rolling = shift(rolling_filled, group_col, date_col,
                                        lag)

                        method_name = method.__name__ if type(
                            method) != str else method

                        new_names = {x: "{0}_lag{1}d_w{2}_key{3}_preag{4}_ag{5}_{6}_dynamic_rolling".\
                                     format(x, lag, w, '_'.join(id_cols), preagg, method_name, filter_col) for x in target_cols}

                        out_df = pd.merge(out_df,
                                          rolling.rename(columns=new_names),
                                          how='left',
                                          on=group_col)
                        # progress.value += 1

    return out_df

In [ ]:
def add_lagged_and_days_info(data):
    data['Promo'] = data['Promo'].fillna(0)
    data['Price'] = data['Regular_Price'] * (1 - data['Promo']) + data['Promo_Price'].fillna(0) * data['Promo']
    data.drop(['Promo_Price'], axis=1, inplace=True)

    data['Date'] = pd.to_datetime(data['Date'])
    data["weekday"] = data['Date'].dt.weekday
    data["monthday"] = data['Date'].dt.day
    data['year'] = data['Date'].dt.year
    data['is_weekend'] = data['weekday'].isin([5,6]) * 1

    target_cols = ['Demand']
    id_cols = ['SKU_id']
    date_col = 'Date'

    return generate_lagged_features(data, 
                                    target_cols = target_cols, 
                                    id_cols = id_cols, 
                                    date_col = date_col, 
                                    lags = [28, 35], 
                                    windows = ['14D', '21D', '28D', '56D'], 
                                    # preagg_methods = ['sum', 'mean', 'count'],
                                    preagg_methods = ['sum'],
                                    agg_methods = ['mean', percentile(10),  percentile(90)],
                                    dynamic_filters = ['Promo', 'is_weekend'],
                                    ewm_params={'Promo': [28], 'is_weekend': [28]}
                                    )

In [ ]:
transformed = add_lagged_and_days_info(data)
transformed.set_index('Date', inplace = True)
transformed.columns

Index(['Store_id', 'SKU_id', 'Promo', 'Demand', 'Regular_Price', 'id', 'Price',
       'weekday', 'monthday', 'year', 'is_weekend',
       'Demand_lag28d_alpha28_keySKU_id_preagsum_Promo_dynamic_ewm',
       'Demand_lag28d_w14D_keySKU_id_preagsum_agmean_Promo_dynamic_rolling',
       'Demand_lag28d_w14D_keySKU_id_preagsum_agpctl10_Promo_dynamic_rolling',
       'Demand_lag28d_w14D_keySKU_id_preagsum_agpctl90_Promo_dynamic_rolling',
       'Demand_lag28d_w21D_keySKU_id_preagsum_agmean_Promo_dynamic_rolling',
       'Demand_lag28d_w21D_keySKU_id_preagsum_agpctl10_Promo_dynamic_rolling',
       'Demand_lag28d_w21D_keySKU_id_preagsum_agpctl90_Promo_dynamic_rolling',
       'Demand_lag28d_w28D_keySKU_id_preagsum_agmean_Promo_dynamic_rolling',
       'Demand_lag28d_w28D_keySKU_id_preagsum_agpctl10_Promo_dynamic_rolling',
       'Demand_lag28d_w28D_keySKU_id_preagsum_agpctl90_Promo_dynamic_rolling',
       'Demand_lag28d_w56D_keySKU_id_preagsum_agmean_Promo_dynamic_rolling',
       'Demand_la

In [ ]:
date = transformed[np.isnan(transformed['id'])].index.unique().max() - pd.Timedelta('28D')
not_test = transformed[np.isnan(transformed['id'])]
train = transformed[(transformed.index < date) & np.isnan(transformed['id'])]
validate = transformed[(transformed.index >= date) & np.isnan(transformed['id'])]
test = transformed[pd.notna(transformed['id'])]

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

errors = list()
ids = list()
results = list()

sku_values = transformed['SKU_id'].unique()
promo_values = transformed['Promo'].unique()

def objective(trial):
    lr = trial.suggest_float('learning_rate', 0.1, 1)
    depth = trial.suggest_int('max_depth', 8, 20)
    subsample = trial.suggest_float('subsample', 0.01, 0.9)
    model = lgb.LGBMRegressor(n_estimators=700, learning_rate=lr, metric='mae', max_depth=depth, subsample=subsample)

    model.fit(X=x_train, y=y_train)
    score = mean_absolute_error(y_validate, model.predict(x_validate))
    return score

for sku in sku_values:
  for promo in promo_values:
      filter = (train['SKU_id'] == sku) & (train['Promo'] == promo) 
      x_train = train[filter].drop(['SKU_id', 'Demand', 'id'], axis=1)
      y_train = train[filter].Demand

      filter = (validate['SKU_id'] == sku) & (validate['Promo'] == promo) 
      x_validate = validate[filter].drop(['SKU_id', 'Demand', 'id'], axis=1)
      y_validate = validate[filter].Demand

      optuna.logging.disable_propagation()
      optuna.logging.disable_default_handler()

      study = optuna.create_study()
      study.optimize(objective, n_trials=20)
      best_params = study.best_params
      print(best_params)

      lgbm = lgb.LGBMRegressor(
                          n_estimators=700,
                          max_depth=best_params['max_depth'],
                          learning_rate=best_params['learning_rate'],
                          #feature_fraction=0.7,
                          subsample=best_params['subsample'],
                          metric='mae')
      
      filter = (not_test['SKU_id'] == sku) & (not_test['Promo'] == promo)
      x_not_test = not_test[filter].drop(['SKU_id', 'Demand', 'id'], axis=1)
      y_not_test = not_test[filter].Demand

      lgbm.fit(X=x_not_test, y=y_not_test)
      # errors.append(mean_absolute_error(lgbm.predict(x_validate), y_validate))

      filter = (test['SKU_id'] == sku) & (test['Promo'] == promo)
      x_test = test[filter]
      
      ids = np.append(ids, x_test['id'])
      results = np.append(results, lgbm.predict(x_test.drop(['SKU_id', 'Demand', 'id'], axis=1)))

{'learning_rate': 0.11511789917495963, 'max_depth': 20, 'subsample': 0.03409868740394867}
{'learning_rate': 0.12971062398318514, 'max_depth': 10, 'subsample': 0.7192113697022455}
{'learning_rate': 0.15976038341492024, 'max_depth': 13, 'subsample': 0.5240157260062642}
{'learning_rate': 0.10160863928510434, 'max_depth': 20, 'subsample': 0.5864166244256457}


In [ ]:
res_pd = pd.DataFrame(columns=['id', 'Demand'])
res_pd['id'] = list(map(int, ids))
res_pd['Demand'] = list(map(np.abs, results))
res_pd.to_csv('results.csv', index=False)  
files.download('results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>